In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import random
import numpy as np

def add_noise(img):
    arr = np.array(img)

    # Gaussian noise
    noise = np.random.normal(0, 25, arr.shape)
    arr = arr + noise

    arr = np.clip(arr, 0, 255).astype(np.uint8)
    return Image.fromarray(arr)

def add_lines(img):
    draw = ImageDraw.Draw(img)
    w, h = img.size

    for _ in range(12):
        x1, y1 = random.randint(0,w), random.randint(0,h)
        x2, y2 = random.randint(0,w), random.randint(0,h)
        draw.line((x1,y1,x2,y2), fill=(0,0,0), width=2)

    return img

def distort(img):
    return img.transform(
        img.size,
        Image.AFFINE,
        (1, random.uniform(-0.3,0.3), 0, random.uniform(-0.3,0.3), 1, 0),
        Image.BILINEAR
    )

img = Image.open("1.png")

img = distort(img)
img = add_noise(img)
img = add_lines(img)
img = img.filter(ImageFilter.GaussianBlur(1))

img.save("ctf_protected.png")


In [ ]:
from PIL import Image, ImageFilter, ImageEnhance
import numpy as np

# Load image
img = Image.open("ctf_protected.png").convert("RGB")

# --- STEP 1: Reduce noise ---
img = img.filter(ImageFilter.MedianFilter(size=3))
img = img.filter(ImageFilter.GaussianBlur(radius=0.8))

# --- STEP 2: Sharpen edges ---
img = img.filter(ImageFilter.UnsharpMask(
    radius=2,
    percent=180,
    threshold=3
))

# --- STEP 3: Boost contrast ---
img = ImageEnhance.Contrast(img).enhance(2.5)
img = ImageEnhance.Sharpness(img).enhance(2.0)

# --- STEP 4: Convert to grayscale + threshold ---
gray = img.convert("L")

arr = np.array(gray)
threshold = arr.mean()
binary = (arr > threshold).astype(np.uint8) * 255

final = Image.fromarray(binary)

final.save("ctf_recovered.png")


In [ ]:
from PIL import Image

def embed_binary_text(image_path, output_path):
    # 🔹 Take binary input from user
    binary_text = input("Enter binary text (only 0s and 1s): ").strip()

    # ✅ Validate input
    if not all(bit in "01" for bit in binary_text):
        print("❌ Invalid input! Use only 0 and 1.")
        return

    img = Image.open("ctf_protected.png").convert("RGB")
    pixels = list(img.getdata())

    # 🔚 End delimiter (16 bits)
    binary_text += "1111111111111110"

    bit_index = 0
    new_pixels = []

    for pixel in pixels:
        r, g, b = pixel

        if bit_index < len(binary_text):
            r = (r & ~1) | int(binary_text[bit_index])
            bit_index += 1
        if bit_index < len(binary_text):
            g = (g & ~1) | int(binary_text[bit_index])
            bit_index += 1
        if bit_index < len(binary_text):
            b = (b & ~1) | int(binary_text[bit_index])
            bit_index += 1

        new_pixels.append((r, g, b))

    if bit_index < len(binary_text):
        print("❌ Image too small to hold this data.")
        return

    img.putdata(new_pixels)
    img.save(output_path)
    print("✅ Binary data embedded successfully!")

# ▶ Run
embed_binary_text("ctf_protected.png", "output.png")


In [ ]:
from PIL import Image

def extract_binary_text(image_path):
    img = Image.open(image_path)
    pixels = list(img.getdata())

    binary_data = ""

    for pixel in pixels:
        for channel in pixel:
            binary_data += str(channel & 1)

            # 🔚 Stop at delimiter
            if binary_data.endswith("1111111111111110"):
                return binary_data[:-16]

    return binary_data

# ▶ Run
hidden_binary = extract_binary_text("output.png")
print("🔓 Extracted Binary:", hidden_binary)
